In [1]:
from math import *
import numpy as np
import cmath
import matplotlib.pyplot as plt
import time
import plotly.graph_objects as go

In [2]:
def sinc(x):
    '''
    Compute the sinc-function: sin(x) / x
    '''
    try:  
        return sin(x) / float(x)
    except ZeroDivisionError:
        return 1.

In [42]:
def upperRecurency(value, Nmax):
    
    # Nmax is the larger index of requirement functions but quantity 
    # of required functions is (Nmax + 1) cause 0-index is included
    
    # Initialization the zero-arrays for calculating the functions
    sphj, sphy, new_sphj = np.zeros(Nmax + 1), np.zeros(Nmax + 1), np.zeros(Nmax + 101)
    
    # Check for non-zero value
    if value:
        
        # Compute the spherical Bessels functions of the first kind of zero 
        # and first-indexes in the current value
        sphj[0] = sinc(value)                                                       # n = 0
        sphj[1] = (-1) * cos(value) / float(value) + sin(value) / float(value ** 2) # n = 1
        
        # Compute the spherical Bessels functions of the second kind of zero
        # and first-indexes in the current value
        sphy[0] = (-1) * cos(value) / float(value)                                  # n = 0
        sphy[1] = (-1) * sin(value / float(value) - cos(value) / float(value ** 2)) # n = 1  
        

    # Compute the spherical Bessels functions of the second kind another indexes
    # with bottom-to-top recurrency relation
    for ind in range(2, Nmax + 1):
        sphy[ind] = (2 * ind - 1) / float(value) * sphy[ind - 1] - sphy[ind - 2]
        
        # For another kind of spherical Bessels functions the previous algorithm isn't stable
        # and we calculate only the part of necessary functions
        if (ind <= value):
            sphj[ind] = (2 * ind - 1) / float(value) * sphj[ind - 1] - sphj[ind - 2]
    
    
    # For another part we should use the top-to-bottom recurrency relation
    
    # Initialization {Nmax + 101} and {Nmax + 100}-indexes for this relation.
    # We should extend our quantity of calculating functions by 100
    # After that our 'new' spherical Bessels functions of the first kind will be known,
    # but it will be the real functions multiplied by the some const.
    new_sphj[Nmax + 100] = 0 # n = Nmax + 101
    new_sphj[Nmax + 99] = 1 # n = Nmax + 100
    
    # Calculate the 'new' spherical Bessels functions of the first kind of {98 + Nmax} down to {0} - indexes
    for ind in range(98 + Nmax, -1, -1):
        new_sphj[ind] = (2 * ind + 3) / float(value) * new_sphj[ind + 1] - new_sphj[ind + 2]
        
        # We shouldn't forget about the fact that our 'new' functions dramatically increases step-by-step.
        # To avoid overflowing, we reduce all of the previous caltulating functions by division on 100
        if new_sphj[ind] > 100:
            
            # We don't care about indexes which we don't want to know
            if ind <= Nmax:
                for index in range(ind, Nmax, 1):
                    new_sphj[index] /= 100
                    
            # For this case we should reduce only the {current index + 1} and {current index} functions
            # to continue our calculation
            else:
                new_sphj[ind + 1] /= 100
                new_sphj[ind] /= 100
                
    # After all calculations we want to know the const of relation between the 'new'
    # and known Bessels functions of the first kind
    for ind in range(len(sphj)):
        
        # Check for zero division to avoid the problem
        if sphj[ind] != 0:
            k = new_sphj[ind] / float(sphj[ind])
            break
    
    # Initialize the new array for recalculation our 'new' Bessels functions with known constant
    bessels = []
    
    # Division all the 'new' functions by known constant
    for elem in new_sphj:
        bessels.append(elem / float(k))
#         print(elem / float(k))
    
#     # Check the top-to-bottom and bottom-to-top recurrency relations
#     for ind in range(len(sphj)):
#         if sphj[ind]:
#             print(f'top_to_bottom, j_{ind}({value}) = {bessels[ind]}\n', f'bottom_to_top, j_{ind}({value}) = {sphj[ind]}\n')
    
    
    # After all kinds of calculations, we return the 1st and 2nd kind of functions as 2 arrays in the current value
    return bessels, sphy
    
# print(upperRecurency(20, 10))

In [95]:
# Square root + theorem of cos
def sqrthcos(r, z0, costheta):
    return (r ** 2 + z0 ** 2 + 2 * r * z0 * costheta) ** .5

# Initialize the parameters of required functions
# Nmax -> quantity of required functionsp
# z0 -> initial coordinate of the spherical-wave sourse
# step -> cells-step
Nmax = 10
z0 = 50
step = 1

# Initialize the matrix 401 x 401 ([-200, 200] x [-200, 200]) with complex data type for theoretical functions
p_ith = np.zeros((401, 401), dtype=complex)
p_spth = np.zeros((401, 401), dtype=complex)

# Initialize the matrix 1 x Nmax for Legendre Polynomials
P = np.zeros(Nmax)

# Initialize the matrix 401 x 401 ([-200, 200] x [-200, 200]) with complex data type for calculating functions
p_i = np.zeros((401, 401), dtype=complex)
p_sp = np.zeros((401, 401), dtype=complex)

# Calculating the Bessels functions with the help of our function called as 'upperRecurency' in the value z0
sphjz0, sphyz0 = upperRecurency(z0, Nmax)

t = time.time()
e = np.zeros((401, 401), dtype=float)

# Going through the cells
for x in range(-200, 201):
    print(x / 4 + 50, '%', '\r', end='')
    for z in range(-200, 201):
        
        # Calculating the radius-vector in the current point
        r = step * (x ** 2 + z ** 2) ** .5
        
        # Check for non-zero value for r
        if r:
            
            # Calculating cos(theta) in the current point
            costheta = step * z / r
            
                
            # Calculating the Bessels functions with the help of our function called as 'upperRecurency' in the value r
            sphj, sphy = upperRecurency(r, Nmax)
                
            # Calculating the theoretical pressure functions
            p_ith[x + 200][z + 200] = cmath.exp(1j * r * costheta)
                
            # Check for non-zero value for theorem of cos in the current point
            if sqrthcos(r, z0, costheta):
                p_spth[x + 200][z + 200] = cmath.exp(1j * sqrthcos(r, z0, costheta)) / sqrthcos(r, z0, costheta)
                
            # Use bottom-to-top recurrency relation to calculate Legendre's polynomials
            P[0] = 1
            P[1] = costheta
            for n in range(0, Nmax - 2):
                P[n + 2] = (2 * n + 1) / (n + 1) * costheta * P[n + 1] - n / (n + 1) * P[n];
                    
            # Calculating the required pressure functions
            for n in range(0, Nmax):
                p_i[x + 200, z + 200] += (1j ** n)  * (2 * n + 1) * P[n] * sphj[n];
                    
                ######################>hankel vmesto sphj<##################
                p_sp[x + 200, z + 200] += 1j * (-1) ** n * (2 * n + 1) * (sphjz0[n] + 1j * sphyz0[n]) * P[n] * sphj[n] 
                
            e[x + 200, z + 200] = abs(p_i[x + 200, z + 200] - p_ith[x + 200, z + 200])
                
print(time.time() - t)

49.09409999847412


In [101]:
x = np.arange(-200, 200, 1)
y = np.arange(-200, 200, 1)
xgrid, ygrid = np.meshgrid(x, y)
zgrid = e[xgrid + 200, ygrid + 200]
for ind in range(len(zgrid)):
    for sec_ind in range(len(zgrid[ind])):
#         print(zgrid[ind][sec_ind])
        if zgrid[ind][sec_ind] > 1:
#             print(p_i[ind][sec_ind], p_ith[ind][sec_ind])
            zgrid[ind][sec_ind] = 1

In [103]:
# fig = go.Figure(data=[go.Surface(z=zgrid, x=xgrid, y=ygrid)])
# fig.update_layout(title='Error Function', autosize=False,
#                   width=1000, height=1000,
#                   margin=dict(l=65, r=50, b=65, t=90))
# fig.show()

In [104]:
z2 = np.zeros((401, 401), dtype=float)
for ind in range(len(p_i)):
    for sec_ind in range(len(p_i[ind])):
#         if abs(p_i[ind][sec_ind] - p_ith[ind][sec_ind]) <= 1.:
        z2[ind][sec_ind] = abs(p_i[ind][sec_ind] - p_ith[ind][sec_ind])
#         else:
#             z2[ind][sec_ind] = 1.

# x, y = np.linspace(-200, 200, 401), np.linspace(-200, 200, 401)
x = np.arange(-200, 200, 1)
y = np.arange(-200, 200, 1)
xgrid, ygrid = np.meshgrid(x, y)
zgrid = z2[xgrid, ygrid]
# fig = go.Figure(data=[go.Surface(z=zgrid, x=xgrid, y=ygrid)])
# fig.update_layout(title='Error Function', autosize=False,
#                   width=1000, height=1000,
#                   margin=dict(l=65, r=50, b=65, t=90))
# fig.show()

In [105]:
z_sp = np.zeros((401, 401), dtype=float)
for ind in range(len(p_sp)):
    for sec_ind in range(len(p_sp[ind])):
        if abs(p_sp[ind][sec_ind] - p_spth[ind][sec_ind]) <= 1:
            z_sp[ind][sec_ind] = abs(p_sp[ind][sec_ind] - p_spth[ind][sec_ind])
        else:
            z_sp[ind][sec_ind] = 1
            
# fig = go.Figure(data=[go.Surface(z=z_sp, x=x, y=y)])
# fig.update_layout(title='Mt Bruno Elevation', autosize=False,
#                   width=1000, height=1000,
#                   margin=dict(l=65, r=50, b=65, t=90))
# fig.show()


#